In [2]:
import numpy as np
import cv2
import pandas as pd
from skimage.filters import roberts, sobel, scharr, prewitt
from scipy import ndimage as nd

In [8]:
def feature_extraction(img):
    # Create dataframe and engineer features
    df = pd.DataFrame()

    # Feature: original pixel values
    img2 = img.reshape(-1)
    df["Original Image"] = img2  
    
    # Feature: first set of Gabor features by applying filters to original image
    # Apply 32 generated Gabor filters w/ various parameters to image 
    num = 1
    kernels = []
    for theta in range(2):
        theta = theta / 4. * np.pi
        for sigma in (1, 3):
            for lamda in np.arange(0, np.pi, np.pi / 4):
                for gamma in (0.05, 0.5):
                    gabor_label = "Gabor" + str(num)
                    ksize = 5
                    kernel = cv2.getGaborKernel(
                        (ksize, ksize), 
                        sigma, theta, lamda, gamma, 0, 
                        ktype=cv2.CV_32F)
                    kernels.append(kernel)
                    fimg = cv2.filter2D(img2, cv2.CV_8UC3, kernel)
                    filtered_img = fimg.reshape(-1)
                    df[gabor_label] = filtered_img
                    print(gabor_label, ": theta=", theta, ": sigma=", sigma,
                         ": lamda=", lamda, ": gamma=", gamma)
                    num += 1
                    
    # Canny edge; edge detection filter
    edges = cv2.Canny(img, 100, 200)
    edges1 = edges.reshape(-1)
    df["Canny Edge"] = edges1
    
    # Some more filters
    edge_roberts = roberts(img)
    edge_roberts1 = edge_roberts.reshape(-1)
    df["Roberts"] = edge_roberts1

    edge_sobel = sobel(img)
    edge_sobel1 = edge_sobel.reshape(-1)
    df["Sobel"] = edge_sobel1

    edge_scharr = scharr(img)
    edge_scharr1 = edge_scharr.reshape(-1)
    df["Scharr"] = edge_scharr1

    edge_prewitt = prewitt(img)
    edge_prewitt1 = edge_prewitt.reshape(-1)
    df["Prewitt"] = edge_prewitt1
    
    gaussian_img = nd.gaussian_filter(img, sigma=3)
    gaussian_img1 = gaussian_img.reshape(-1)
    df["Gaussian s3"] = gaussian_img1

    gaussian_img2 = nd.gaussian_filter(img, sigma=7)
    gaussian_img3 = gaussian_img2.reshape(-1)
    df["Gaussian s7"] = gaussian_img3

    median_img = nd.median_filter(img, size=3)
    median_img1 = median_img.reshape(-1)
    df["Median s3"] = median_img1
    
    return df

In [9]:
import glob
import pickle
from matplotlib import pyplot as plt

In [12]:
filename = "sandstone_model"
load_model = pickle.load(open(filename, "rb"))

path = "Train_images/*.tif"
for file in glob.glob(path):
    img1 = cv2.imread(file)
    img = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    
    X = feature_extraction(img)
    result = load_model.predict(X)
    segmented = result.reshape((img.shape))
    
    name = file.split("e_")
    
    plt.imsave("rf_segmented/" + name[1] + ".jpg", segmented, cmap="turbo")

Gabor1 : theta= 0.0 : sigma= 1 : lamda= 0.0 : gamma= 0.05
Gabor2 : theta= 0.0 : sigma= 1 : lamda= 0.0 : gamma= 0.5
Gabor3 : theta= 0.0 : sigma= 1 : lamda= 0.7853981633974483 : gamma= 0.05
Gabor4 : theta= 0.0 : sigma= 1 : lamda= 0.7853981633974483 : gamma= 0.5
Gabor5 : theta= 0.0 : sigma= 1 : lamda= 1.5707963267948966 : gamma= 0.05
Gabor6 : theta= 0.0 : sigma= 1 : lamda= 1.5707963267948966 : gamma= 0.5
Gabor7 : theta= 0.0 : sigma= 1 : lamda= 2.356194490192345 : gamma= 0.05
Gabor8 : theta= 0.0 : sigma= 1 : lamda= 2.356194490192345 : gamma= 0.5
Gabor9 : theta= 0.0 : sigma= 3 : lamda= 0.0 : gamma= 0.05
Gabor10 : theta= 0.0 : sigma= 3 : lamda= 0.0 : gamma= 0.5
Gabor11 : theta= 0.0 : sigma= 3 : lamda= 0.7853981633974483 : gamma= 0.05
Gabor12 : theta= 0.0 : sigma= 3 : lamda= 0.7853981633974483 : gamma= 0.5
Gabor13 : theta= 0.0 : sigma= 3 : lamda= 1.5707963267948966 : gamma= 0.05
Gabor14 : theta= 0.0 : sigma= 3 : lamda= 1.5707963267948966 : gamma= 0.5
Gabor15 : theta= 0.0 : sigma= 3 : lamda= 2

Gabor17 : theta= 0.7853981633974483 : sigma= 1 : lamda= 0.0 : gamma= 0.05
Gabor18 : theta= 0.7853981633974483 : sigma= 1 : lamda= 0.0 : gamma= 0.5
Gabor19 : theta= 0.7853981633974483 : sigma= 1 : lamda= 0.7853981633974483 : gamma= 0.05
Gabor20 : theta= 0.7853981633974483 : sigma= 1 : lamda= 0.7853981633974483 : gamma= 0.5
Gabor21 : theta= 0.7853981633974483 : sigma= 1 : lamda= 1.5707963267948966 : gamma= 0.05
Gabor22 : theta= 0.7853981633974483 : sigma= 1 : lamda= 1.5707963267948966 : gamma= 0.5
Gabor23 : theta= 0.7853981633974483 : sigma= 1 : lamda= 2.356194490192345 : gamma= 0.05
Gabor24 : theta= 0.7853981633974483 : sigma= 1 : lamda= 2.356194490192345 : gamma= 0.5
Gabor25 : theta= 0.7853981633974483 : sigma= 3 : lamda= 0.0 : gamma= 0.05
Gabor26 : theta= 0.7853981633974483 : sigma= 3 : lamda= 0.0 : gamma= 0.5
Gabor27 : theta= 0.7853981633974483 : sigma= 3 : lamda= 0.7853981633974483 : gamma= 0.05
Gabor28 : theta= 0.7853981633974483 : sigma= 3 : lamda= 0.7853981633974483 : gamma= 0.5


Gabor28 : theta= 0.7853981633974483 : sigma= 3 : lamda= 0.7853981633974483 : gamma= 0.5
Gabor29 : theta= 0.7853981633974483 : sigma= 3 : lamda= 1.5707963267948966 : gamma= 0.05
Gabor30 : theta= 0.7853981633974483 : sigma= 3 : lamda= 1.5707963267948966 : gamma= 0.5
Gabor31 : theta= 0.7853981633974483 : sigma= 3 : lamda= 2.356194490192345 : gamma= 0.05
Gabor32 : theta= 0.7853981633974483 : sigma= 3 : lamda= 2.356194490192345 : gamma= 0.5
Gabor1 : theta= 0.0 : sigma= 1 : lamda= 0.0 : gamma= 0.05
Gabor2 : theta= 0.0 : sigma= 1 : lamda= 0.0 : gamma= 0.5
Gabor3 : theta= 0.0 : sigma= 1 : lamda= 0.7853981633974483 : gamma= 0.05
Gabor4 : theta= 0.0 : sigma= 1 : lamda= 0.7853981633974483 : gamma= 0.5
Gabor5 : theta= 0.0 : sigma= 1 : lamda= 1.5707963267948966 : gamma= 0.05
Gabor6 : theta= 0.0 : sigma= 1 : lamda= 1.5707963267948966 : gamma= 0.5
Gabor7 : theta= 0.0 : sigma= 1 : lamda= 2.356194490192345 : gamma= 0.05
Gabor8 : theta= 0.0 : sigma= 1 : lamda= 2.356194490192345 : gamma= 0.5
Gabor9 : the